In [ ]:
#Timestamps are collected at various stages to understand processing speed.
# prompt: print date and time stamp with milliseconds

import datetime

def current_datetime_millis():
  now = datetime.datetime.now()
  millis = int(round(now.timestamp() * 1000))
  return now.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]


print(current_datetime_millis())

2024-11-13 04:05:30.673


In [ ]:
!pip install matplotlib
!pip install Image
!pip install openpyxl
!pip install pynvml
!pip install tensorflow_addons

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.1 MB/s eta 0:00:00
  Created wheel for Image: filename=image-1.5.33-py2.py3-none-any.whl size=19482 sha256=d05adab8f79094f0ded7737252074a64df44467c79128f643fc043dbbb5d9bba
  Stored in directory: /root/.cache/pip/wheels/70/0c/a4/7cfa53a5c6225c2db2bfec08e782b43d0f25fdae2e995b69be
Successfully built Image
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.1 MB/s eta 0:00:00


In [ ]:

import copy
import os
import random
import shutil
import zipfile
from math import atan2, cos, sin, sqrt, pi, log

import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from numpy import linalg as LA
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, losses, datasets
import keras
%matplotlib inline
import pynvml
# for bulding and running deep learning model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D, ZeroPadding1D, ZeroPadding3D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

#Device Setup

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Check if TensorFlow is using the GPU
if tf.test.is_gpu_available():
    print("TensorFlow is using the GPU")

    # Initialize the pynvml library
    pynvml.nvmlInit()

    # Get the number of GPU devices
    num_gpus = pynvml.nvmlDeviceGetCount()

    # Iterate over GPU devices
    for i in range(num_gpus):
        # Get the device identifier
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        # Get the full GPU name
        gpu_name = pynvml.nvmlDeviceGetName(handle)
        print("GPU Name:", gpu_name)

    # Shutdown the pynvml library
    pynvml.nvmlShutdown()
else:
    print("TensorFlow is not using the GPU")

TensorFlow is not using the GPU


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [ ]:
gpus

[]

In [ ]:
#Prevent out of memory error
#for gpu in gpus:
 #   tf.config.experimental.set_memory_growth(gpu,True)

# Data Import From CVS/XLS



In [ ]:
import pandas

In [ ]:
!pip install -q xlrd
!pip install openpyxl
import pandas
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 1.3 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/SegmentationList.xlsx"

In [ ]:
df = pandas.read_excel(path, 7, header=None, names=['masks','images','split'])

#Data Processing

In [ ]:
trainData = df.loc[df['split'] == 'TRAIN']
testData = df.loc[df['split'] == 'TEST']

In [ ]:
totallen = len(df)
trainlen = len(trainData)
testlen = len(testData)
print(totallen)
print(trainlen)
print(testlen)

10000
9000
1000


In [ ]:
# prompt: while training percentage is greater than 80, split the dataset by .01%

while (trainlen / totallen) * 100 > 80:
  trainData, splitTrained = train_test_split(trainData, test_size=0.0001)
  testData = pandas.concat([testData, splitTrained])
  trainlen = len(trainData)
  testlen = len(testData)
print("Training percentage:", (trainlen / totallen) * 100)
print("Testing percentage:", (testlen / totallen) * 100)


Training percentage: 80.0
Testing percentage: 20.0


In [ ]:
# prompt: while training percentage is greater than 80, split the dataset by .01%

while (testlen / totallen) * 100 > 20:
  testData, splitTest = train_test_split(testData, test_size=0.0001)
  trainData = pandas.concat([trainData, splitTest])
  trainlen = len(trainData)
  testlen = len(testData)
print("Training percentage:", (trainlen / totallen) * 100)
print("Testing percentage:", (testlen / totallen) * 100)

Training percentage: 80.0
Testing percentage: 20.0


In [ ]:
train_image_paths = trainData['images'].values
train_mask_paths = trainData['masks'].values

In [ ]:
test_image_paths = testData['images'].values
test_mask_paths = testData['masks'].values

In [ ]:
directory = "/content/drive/My Drive/"
replaceString = "/Volumes/Frederick Harris II/Dissertation Data/";

In [ ]:
for i in range(len(train_image_paths)):
    train_image_paths[i] = train_image_paths[i].replace(replaceString, directory)

for i in range(len(train_mask_paths)):
    train_mask_paths[i] = train_mask_paths[i].replace(replaceString, directory)

for i in range(len(test_image_paths)):
    test_image_paths[i] = test_image_paths[i].replace(replaceString, directory)

for i in range(len(test_mask_paths)):
    test_mask_paths[i] = test_mask_paths[i].replace(replaceString, directory)

In [ ]:

print(len(train_image_paths))
print(len(train_mask_paths))
print(train_image_paths[:5])
print(train_mask_paths[:5])

8000
8000
['/content/drive/My Drive/UECFOODPIXCOMPLETE/train/images/4907.jpg'
 '/content/drive/My Drive/UECFOODPIXCOMPLETE/train/images/1657.jpg'
 '/content/drive/My Drive/UECFOODPIXCOMPLETE/train/images/6577.jpg'
 '/content/drive/My Drive/UECFOODPIXCOMPLETE/train/images/7574.jpg'
 '/content/drive/My Drive/UECFOODPIXCOMPLETE/train/images/189.jpg']
['/content/drive/My Drive/UECFOODPIXCOMPLETE/train/masks/4907.png'
 '/content/drive/My Drive/UECFOODPIXCOMPLETE/train/masks/1657.png'
 '/content/drive/My Drive/UECFOODPIXCOMPLETE/train/masks/6577.png'
 '/content/drive/My Drive/UECFOODPIXCOMPLETE/train/masks/7574.png'
 '/content/drive/My Drive/UECFOODPIXCOMPLETE/train/masks/189.png']


In [ ]:
#Remove items without proper exts
for i in train_image_paths:
    if ".jpg" not in i and ".png" not in i:
        print(i)
        train_image_paths.remove(i)

In [ ]:
#Remove items without proper exts
for i in train_mask_paths:
    if ".jpg" not in i and ".png" not in i:
        print(i)
        train_mask_paths.remove(i)

In [ ]:
test_image_paths = sorted(test_image_paths)
test_mask_paths = sorted(test_mask_paths)

In [ ]:
#Remove items without proper exts
for i in test_image_paths:
    if ".jpg" not in i and ".png" not in i:
        print(i)
        test_image_paths.remove(i)

In [ ]:
#Remove items without proper exts
for i in test_mask_paths:
    if ".jpg" not in i and ".png" not in i:
        print(i)
        test_mask_paths.remove(i)

In [ ]:
print(len(test_image_paths))
print(len(test_mask_paths))
print(test_image_paths[:5])
print(test_mask_paths[:5])

2000
2000
['/content/drive/My Drive/UECFOODPIXCOMPLETE/test/images/1004.jpg', '/content/drive/My Drive/UECFOODPIXCOMPLETE/test/images/1005.jpg', '/content/drive/My Drive/UECFOODPIXCOMPLETE/test/images/102.jpg', '/content/drive/My Drive/UECFOODPIXCOMPLETE/test/images/103.jpg', '/content/drive/My Drive/UECFOODPIXCOMPLETE/test/images/1034.jpg']
['/content/drive/My Drive/UECFOODPIXCOMPLETE/test/masks/1004.png', '/content/drive/My Drive/UECFOODPIXCOMPLETE/test/masks/1005.png', '/content/drive/My Drive/UECFOODPIXCOMPLETE/test/masks/102.png', '/content/drive/My Drive/UECFOODPIXCOMPLETE/test/masks/103.png', '/content/drive/My Drive/UECFOODPIXCOMPLETE/test/masks/1034.png']


In [ ]:
import datetime;

# ct stores current time
ts = datetime.datetime.now()
print("current time:-", ts)

current time:- 2024-11-13 04:06:40.979615


In [ ]:
train_y = []
train_x = []

SIZE = 128
img_shape = (SIZE,SIZE,3)
mask_shape = (SIZE,SIZE,1)

##Convert data to numpy array for simpler processng

In [ ]:
progress = 0
progInt = 10
for i in range(len(train_image_paths)):    #Remember enumerate method adds a counter and returns the enumerate object
      image = cv2.imread(train_image_paths[i], 1) #RGB
      image = Image.fromarray(image)
      image = image.resize((SIZE, SIZE))
      train_x.append(np.array(image))

      mask = cv2.imread(train_mask_paths[i], 0)
      mask = Image.fromarray(mask)
      mask = mask.resize((SIZE, SIZE))
      train_y.append(np.array(mask))

      if(100*(i/len(train_image_paths)) >= progress):
        ts = datetime.datetime.now()
        print(ts, " " ,progress, "% (", i, " / ", len(train_image_paths), ")")
        progress += progInt

print(ts, " 100 %")

2024-11-13 04:07:23.152469   0 % ( 0  /  8000 )
2024-11-13 04:14:09.198770   10 % ( 800  /  8000 )
2024-11-13 04:21:02.444161   20 % ( 1600  /  8000 )
2024-11-13 04:27:25.000056   30 % ( 2400  /  8000 )
2024-11-13 04:34:03.741016   40 % ( 3200  /  8000 )
2024-11-13 04:40:28.284999   50 % ( 4000  /  8000 )


In [ ]:
test_y = []
test_x = []

In [ ]:
progress = 0
progInt = 10
for i in range(len(test_image_paths)):    #Remember enumerate method adds a counter and returns the enumerate object
      image = cv2.imread(test_image_paths[i], 1) #RGB
      image = Image.fromarray(image)
      image = image.resize((SIZE, SIZE))
      test_x.append(np.array(image))

      mask = cv2.imread(test_mask_paths[i], 0)
      mask = Image.fromarray(mask)
      mask = mask.resize((SIZE, SIZE))
      test_y.append(np.array(mask))

      if(100*(i/len(test_image_paths)) > progress):
        ts = datetime.datetime.now()
        print(ts, " " ,progress, "% (", i, " / ", len(test_image_paths), ")")
        progress += progInt

print(ts, "  100 %")

In [ ]:
print(current_datetime_millis())

## Data normalization

In [ ]:
#Normalize images
train_x = np.expand_dims(normalize(np.array(train_x), axis=1),3)

#Do not normalize masks, just rescale to 0 to 1.
train_y = np.expand_dims((np.array(train_y)),3) /255.

In [ ]:
#Normalize images
test_x = np.expand_dims(normalize(np.array(test_x), axis=1),3)
#D not normalize masks, just rescale to 0 to 1.
test_y = np.expand_dims((np.array(test_y)),3) /255.

In [ ]:
train_x.shape

In [ ]:
train_x = np.squeeze(train_x, 3)

In [ ]:
train_x.shape

In [ ]:
train_y.shape

In [ ]:
test_x.shape

In [ ]:
test_x = np.squeeze(test_x, 3)

In [ ]:
test_x.shape

In [ ]:
test_y.shape

In [ ]:
SIZE

## Data splitting

In [ ]:
from sklearn.model_selection import train_test_split
X_valid, test_x, Y_valid, test_y = train_test_split(test_x, test_y, test_size = 0.50, random_state = 0)
#X_valid, train_x, Y_valid, train_y = train_test_split(train_x, train_y, test_size = 0.20, random_state = 0)


In [ ]:
# prompt: perform the same shuffle on train_x and train_y

from sklearn.utils import shuffle

train_x, train_y = shuffle(train_x, train_y, random_state=0)
test_x, test_y = shuffle(test_x, test_y, random_state=0)
X_valid, Y_valid = shuffle(X_valid, Y_valid, random_state=0)


In [ ]:
SIZE

#ML Algorithm Setup

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda, Layer
from tensorflow.keras import backend as K

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function


In [ ]:
# DeepLabV3+ Model

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, UpSampling2D
from tensorflow.keras.layers import AveragePooling2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

""" Atrous Spatial Pyramid Pooling """
def ASPP_DLV3(inputs):
    shape = inputs.shape

    y_pool = AveragePooling2D(pool_size=(shape[1], shape[2]), name='average_pooling')(inputs)
    y_pool = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(y_pool)
    y_pool = BatchNormalization(name=f'bn_1')(y_pool)
    y_pool = Activation('relu', name=f'relu_1')(y_pool)
    y_pool = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y_pool)

    y_1 = Conv2D(filters=256, kernel_size=1, dilation_rate=1, padding='same', use_bias=False)(inputs)
    y_1 = BatchNormalization()(y_1)
    y_1 = Activation('relu')(y_1)

    y_6 = Conv2D(filters=256, kernel_size=3, dilation_rate=6, padding='same', use_bias=False)(inputs)
    y_6 = BatchNormalization()(y_6)
    y_6 = Activation('relu')(y_6)

    y_12 = Conv2D(filters=256, kernel_size=3, dilation_rate=12, padding='same', use_bias=False)(inputs)
    y_12 = BatchNormalization()(y_12)
    y_12 = Activation('relu')(y_12)

    y_18 = Conv2D(filters=256, kernel_size=3, dilation_rate=18, padding='same', use_bias=False)(inputs)
    y_18 = BatchNormalization()(y_18)
    y_18 = Activation('relu')(y_18)

    y = Concatenate()([y_pool, y_1, y_6, y_12, y_18])

    y = Conv2D(filters=256, kernel_size=1, dilation_rate=1, padding='same', use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    return y

def DeepLabV3Plus(shape):
    """ Inputs """
    inputs = Input(shape)

    """ Pre-trained ResNet50 """
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    """ Pre-trained ResNet50 Output """
    image_features = base_model.get_layer('conv4_block6_out').output
    x_a = ASPP_DLV3(image_features)
    x_a = UpSampling2D((4, 4), interpolation="bilinear")(x_a)

    """ Get low-level features """
    x_b = base_model.get_layer('conv2_block2_out').output
    x_b = Conv2D(filters=48, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    x = Concatenate()([x_a, x_b])

    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((4, 4), interpolation="bilinear")(x)

    """ Outputs """
    x = Conv2D(1, (1, 1), name='output_layer')(x)
    x = Activation('sigmoid')(x)

    """ Model """
    model = Model(inputs=inputs, outputs=x)
    return model

def DeepLabV3PlusNoWeights(shape):
    """ Inputs """
    inputs = Input(shape)

    """ Pre-trained ResNet50 """
    base_model = ResNet50(weights=None, include_top=False, input_tensor=inputs)

    """ Pre-trained ResNet50 Output """
    image_features = base_model.get_layer('conv4_block6_out').output
    x_a = ASPP_DLV3(image_features)
    x_a = UpSampling2D((4, 4), interpolation="bilinear")(x_a)

    """ Get low-level features """
    x_b = base_model.get_layer('conv2_block2_out').output
    x_b = Conv2D(filters=48, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    x = Concatenate()([x_a, x_b])

    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((4, 4), interpolation="bilinear")(x)

    """ Outputs """
    x = Conv2D(1, (1, 1), name='output_layer')(x)
    x = Activation('sigmoid')(x)

    """ Model """
    model = Model(inputs=inputs, outputs=x)
    return model

if False:
    input_shape = (512, 512, 3)
    model = DeepLabV3Plus(input_shape)
    model.summary()

In [ ]:
#Unet

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block_unet(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block_unet(input, num_filters):
    x = conv_block_unet(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block_unet(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block_unet(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block_unet(inputs, 64)
    s2, p2 = encoder_block_unet(p1, 128)
    s3, p3 = encoder_block_unet(p2, 256)
    s4, p4 = encoder_block_unet(p3, 512)

    b1 = conv_block_unet(p4, 1024)

    d1 = decoder_block_unet(b1, s4, 512)
    d2 = decoder_block_unet(d1, s3, 256)
    d3 = decoder_block_unet(d2, s2, 128)
    d4 = decoder_block_unet(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

if False:
    input_shape = (512, 512, 3)
    model = build_unet(input_shape)
    model.summary()

In [ ]:
# Double UNEt

In [ ]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply, AveragePooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19

def squeeze_excite_block(inputs, ratio=8):
    init = inputs       ## (b, 128, 128, 32)
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)     ## (b, 32)   -> (b, 1, 1, 32)
    se = Reshape(se_shape)(se)
    se = Dense(filters//ratio, activation="relu", use_bias=False)(se)
    se = Dense(filters, activation="sigmoid", use_bias=False)(se)

    x = Multiply()([inputs, se])
    return x

def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def conv_block(x, filters):
    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = []

    model = VGG19(include_top=False, weights="imagenet", input_tensor=inputs)
    names = ["block1_conv2", "block2_conv2", "block3_conv4", "block4_conv4"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("block5_conv4").output
    return output, skip_connections

def encoder1NoWeights(inputs):
    skip_connections = []

    model = VGG19(include_top=False, weights=None, input_tensor=inputs)
    names = ["block1_conv2", "block2_conv2", "block3_conv4", "block4_conv4"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("block5_conv4").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [256, 128, 64, 32]
    skip_connections.reverse()

    x = inputs
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation="bilinear")(x)
        x = Concatenate()([x, skip_connections[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, 1, padding="same")(inputs)
    x = Activation("sigmoid")(x)
    return x

def encoder2(inputs):
    num_filters = [32, 64, 128, 256]
    skip_connections = []

    x = inputs
    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPool2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 32]
    skip_2.reverse()

    x = inputs
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation="bilinear")(x)
        x = Concatenate()([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def build_double_unet_model(input_shape):
    inputs = Input(input_shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    output1 = output_block(x)

    x = inputs * output1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    output2 = output_block(x)

    outputs = Concatenate()([output1, output2])
    model = Model(inputs, outputs)
    return model

def build_double_unet_model_no_weights(input_shape):
    inputs = Input(input_shape)
    x, skip_1 = encoder1NoWeights(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    output1 = output_block(x)

    x = inputs * output1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    output2 = output_block(x)

    outputs = Concatenate()([output1, output2])
    model = Model(inputs, outputs)
    return model

if False:
    input_shape = (256, 256, 3)
    model = build_double_unet_model(input_shape)
    model.summary()

In [ ]:
#ResNet

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Input
from tensorflow.keras.models import Model

def batchnorm_relu_resnet(inputs):
    """ Batch Normalization & ReLU """
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)
    return x

def residual_block_resnet(inputs, num_filters, strides=1):
    """ Convolutional Layers """
    x = batchnorm_relu_resnet(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=strides)(x)
    x = batchnorm_relu_resnet(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1)(x)

    """ Shortcut Connection (Identity Mapping) """
    s = Conv2D(num_filters, 1, padding="same", strides=strides)(inputs)

    """ Addition """
    x = x + s
    return x

def decoder_block_resnet(inputs, skip_features, num_filters):
    """ Decoder Block """

    x = UpSampling2D((2, 2))(inputs)
    x = Concatenate()([x, skip_features])
    x = residual_block_resnet(x, num_filters, strides=1)
    return x

def build_resunet(input_shape):
    """ RESUNET Architecture """

    inputs = Input(input_shape)

    """ Endoder 1 """
    x = Conv2D(64, 3, padding="same", strides=1)(inputs)
    x = batchnorm_relu_resnet(x)
    x = Conv2D(64, 3, padding="same", strides=1)(x)
    s = Conv2D(64, 1, padding="same")(inputs)
    s1 = x + s

    """ Encoder 2, 3 """
    s2 = residual_block_resnet(s1, 128, strides=2)
    s3 = residual_block_resnet(s2, 256, strides=2)

    """ Bridge """
    b = residual_block_resnet(s3, 512, strides=2)

    """ Decoder 1, 2, 3 """
    x = decoder_block_resnet(b, s3, 256)
    x = decoder_block_resnet(x, s2, 128)
    x = decoder_block_resnet(x, s1, 64)

    """ Classifier """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(x)

    """ Model """
    model = Model(inputs, outputs, name="RESUNET")

    return model

if False:
    shape = (224, 224, 3)
    model = build_resunet(shape)
    model.summary()

#ML Algortithm testing

In [ ]:
epoch_num = 30
batch_num = 1
lr = 0.00001


In [ ]:
num_classes = 2

In [ ]:
print(current_datetime_millis())

In [ ]:
from tensorflow.keras.metrics import MeanIoU
writeDir = "/content/drive/My Drive"
outfilepath = 'config_output_3.txt'
iterations = 40

for i in range(iterations):

  #create and compile models
  unet = build_unet(img_shape)
  unet.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr), loss = [jacard_coef_loss], metrics = [jacard_coef])

  resnet = build_resunet(img_shape)
  resnet.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr), loss = [jacard_coef_loss], metrics = [jacard_coef])

  # Create data output
  dataOutput = []



  historyUnet = unet.fit(x=train_x, y=train_y,
                    batch_size = batch_num,
                    verbose=0,
                    epochs=epoch_num,
                    validation_data=(X_valid, Y_valid),
                    shuffle=True)

  accuracy = historyUnet.history['jacard_coef']
  mean_accuracy = np.mean(accuracy)

  val_jacard_coef = historyUnet.history['val_jacard_coef']
  mean_val_jacard_coef = np.mean(val_jacard_coef)

  loss = historyUnet.history['loss']
  mean_loss = np.mean(loss)

  dataOutput.append(mean_accuracy)
  dataOutput.append(mean_val_jacard_coef)
  dataOutput.append(mean_loss)

  # evaluate model
  loss, acc, *other = unet.evaluate(test_x, test_y)
  dataOutput.append(acc)
  dataOutput.append(0)
  dataOutput.append(loss)

  y_pred = unet.predict(test_x)
  y_pred_classes = (y_pred > 0.5).astype(int)
  iou_metric = MeanIoU(num_classes=num_classes)
  iou_metric.update_state(test_y, y_pred_classes)
  iou = iou_metric.result().numpy()
  print("IoU:", iou)
  dataOutput.append(iou)

  historyResnet = resnet.fit(x=train_x, y=train_y,
                    batch_size = batch_num,
                    verbose=0,
                    epochs=epoch_num,
                    validation_data=(X_valid, Y_valid),
                    shuffle=True)

  accuracy = historyResnet.history['jacard_coef']
  mean_accuracy = np.mean(accuracy)

  val_jacard_coef =historyResnet.history['val_jacard_coef']
  mean_val_jacard_coef = np.mean(val_jacard_coef)

  loss = historyResnet.history['loss']
  mean_loss = np.mean(loss)

  dataOutput.append(mean_accuracy)
  dataOutput.append(mean_val_jacard_coef)
  dataOutput.append(mean_loss)

  # evaluate model
  loss, acc, *other = resnet.evaluate(test_x, test_y)
  dataOutput.append(acc)
  dataOutput.append(0)
  dataOutput.append(loss)

  y_pred = resnet.predict(test_x)
  y_pred_classes = (y_pred > 0.5).astype(int)
  iou_metric = MeanIoU(num_classes=num_classes)
  iou_metric.update_state(test_y, y_pred_classes)
  iou = iou_metric.result().numpy()
  dataOutput.append(iou)

  #Save data ouptut
  filename = os.path.join(writeDir, outfilepath)

  if not os.path.exists(filename):
    with open(filename, 'w') as f:
      pass  # Create an empty file if it doesn't exist

  with open(filename, 'a') as f:
    f.write(' '.join(map(str, dataOutput)) + '\n')
    print("Data saved to", filename)
    print(current_datetime_millis())


32/32 [==============================] - 26s 810ms/step
IoU: 0.37506425
